In [124]:
import pandas as pd

In [125]:
pop2009 = pd.read_csv("POP2009.csv")

columns_to_keep1 = ["name", "popestimate2009"]
pop2009 = pop2009[columns_to_keep1]

# Define a dictionary to map old column names to new ones
column_renamed = {
    'NAME': 'name',
    'POPESTIMATE2009 ' : 'popestimate2009',
}

# Rename the columns using the mapping
pop2009.rename(columns=column_renamed, inplace=True)

# Save the DataFrame with the new column names to a new CSV file
pop2009.to_csv('POP2009.csv', index=False)

pop2009.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NAME              53 non-null     object
 1   POPESTIMATE2009   53 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 980.0+ bytes


In [62]:
pop2020 = pd.read_csv("POP2010-2020.csv")

# Assuming nuforc is a Pandas DataFrame
columns_to_delete = ["SUMLEV", "REGION", "DIVISION", "STATE", "CENSUS2010POP", "ESTIMATESBASE2010", "POPESTIMATE042020", "POPESTIMATE2020" ]
pop2020 = pop2020.drop(columns=columns_to_delete)
pop2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   NAME             57 non-null     object
 1   POPESTIMATE2010  57 non-null     int64 
 2   POPESTIMATE2011  57 non-null     int64 
 3   POPESTIMATE2012  57 non-null     int64 
 4   POPESTIMATE2013  57 non-null     int64 
 5   POPESTIMATE2014  57 non-null     int64 
 6   POPESTIMATE2015  57 non-null     int64 
 7   POPESTIMATE2016  57 non-null     int64 
 8   POPESTIMATE2017  57 non-null     int64 
 9   POPESTIMATE2018  57 non-null     int64 
 10  POPESTIMATE2019  57 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 5.0+ KB


In [63]:
pop2022 = pd.read_csv("POP2020-2022.csv")

# Too many columns to delete, choosing a function that keeps certain columns instead
columns_to_keep2 = ["NAME", "POPESTIMATE2020", "POPESTIMATE2021", "POPESTIMATE2022"]
pop2022 = pop2022[columns_to_keep2]

merged_pop = pd.merge(pop2009, pop2020, on="NAME")
merged_pop = pd.merge(merged_pop, pop2022, on="NAME")

# Rename the columns
for column in merged_pop.columns:
    merged_pop.rename(columns={column: column.strip().replace('POPESTIMATE', '')}, inplace=True)


merged_pop.to_csv('POP2009-2022.csv', index=False)


In [64]:
#Make a CVS file with columns containing NAME (of state), UFO sightings(2009-2022)
ufo_data_path = "../nuforc_reports.csv"
udf = pd.read_csv(ufo_data_path)

#Filter rows where the "country" column is equal to "USA" and keep only necessary coloumns
udf = udf[udf['country'] == 'USA']
udf = udf[["state", "country", "date"]]

# Convert the "date_time" column to datetime and extract year
udf['date'] = pd.to_datetime(udf['date'])
udf['year'] = udf['date'].dt.year

# Filter the DataFrame to include only the years between 2000 and 2010
udf_filtered = udf[ (udf['year'] >= 2009) & (udf['year'] <= 2022)]

# Group and aggregate data to calculate sightings totals per state and year
stateSightings = udf_filtered.groupby(['state', 'year']).size().reset_index(name='sightings')
countrySightings = udf_filtered.groupby(['country', 'year']).size().reset_index(name='sightings')


#pivots table and cleans column and
def pivotTable(place, sightings):
    pvPLACE = sightings.pivot(index=place, columns='year', values='sightings')
    pvPLACE.reset_index(inplace=True)
    pvPLACE.columns.name = None

    # Fill any missing values (NaN) with zeros if necessary
    pvPLACE.fillna(0, inplace=True)

    pvPLACE.to_csv(f'{place}sightings2009-2022.csv', index=False)

pivotTable('country', countrySightings)
pivotTable('state', stateSightings)



In [65]:
#Merge both table
state_data_path = "statesightings2009-2022.csv"
country_data_path = "countrysightings2009-2022.csv"

sdf = pd.read_csv(state_data_path)
cdf = pd.read_csv(country_data_path)

#Ensure clean merging
cdf = cdf.rename(columns={'country': 'NAME'})
sdf = sdf.rename(columns={'state': 'NAME'})

USAdf = pd.concat([sdf, cdf], ignore_index=True)
pdf = pd.read_csv("POP2009-2022.csv")

# Mapping of state abbreviations to full names
state_abbreviations = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'USA': 'United States'
}


# Clean and transform the 'state' column
USAdf['NAME'] = USAdf['NAME'].str.upper().str.strip().map(state_abbreviations)
sdf.dropna(subset=['NAME'], inplace=True)  

# List of American state names
american_states = set(state_abbreviations.values())

# Filter the DataFrame to include only American states
USAdf = USAdf[USAdf['NAME'].isin(american_states)]
USAdf = USAdf.groupby('NAME', as_index=False).sum()
pdf = pdf.groupby('NAME', as_index=False).sum()


USAdf.to_csv('USAsightings2009-2022.csv', index=False)

In [66]:
# Set the 'NAME' column as the index for both DataFrames
pdf.set_index('NAME', inplace=True)
USAdf.set_index('NAME', inplace=True)

# Perform the operation df1 / df2 * 100000
SCALEDdf = ((USAdf / pdf) * 1000000).round(0)

SCALEDdf.dropna(how='all', inplace=True)
SCALEDdf.to_csv('SCALEDsightings2009-2022.csv')
